In [59]:
from os import path
import speech_recognition as sr
from pydub import AudioSegment
import scipy.io.wavfile as wav
import wave
import contextlib

import time
from IPython import display
import ffmpeg

In [2]:
def length_wav(path_file):
    fname = path_file
    with contextlib.closing(wave.open(fname,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        length = frames / float(rate)
        return int(length)

In [3]:
def get_transcript(path_file):

    r = sr.Recognizer()
    audio = sr.AudioFile(path_file)
    
    length = length_wav(path_file)
    num_parts = length / 200

    if length < 200:
        with audio as source:
            audio = r.adjust_for_ambient_noise(source)
            audio = r.record(source)
        transcript = r.recognize_google(audio)
    else:
        arr1 = []
        arr2 = []
        for num in range(1, round(num_parts) + 1):
            arr1.append(f"audio_{num}")


        with audio as source:
            for num in range(0, len(arr1)):
                arr1[num] = r.adjust_for_ambient_noise(source)
                arr1[num] = r.record(source, duration = 200)
                arr2.append(r.recognize_google(arr1[num]))

        transcript = ""
        for i in range(0, len(arr2)):
            transcript = transcript + f" {arr2[i]}"

    return transcript

In [4]:
def get_mic(silence_length):
    r = sr.Recognizer()
    r.pause_threshold = silence_length

    mic = sr.Microphone()

    with mic as source:
        audio = r.adjust_for_ambient_noise(source, duration = 0.5)
        audio = r.listen(source)
    
    with open("data/microphone_results.wav", "wb") as f:
        f.write(audio.get_wav_data())
        
    transcript = get_transcript("data/microphone_results.wav")
    
    return transcript